# Evaluación del modelo Xception para el Tamizaje automatizado de glaucoma con Inteligencia Artificial

# 3. Models

Predicción de Glaucoma con bases públicas ORIGA, REFUGE y G1020.
Xception (Extreme Inception) es una arquitectura de red neuronal convolucional profunda propuesta por François Chollet en 2017. Parte de la familia Inception, pero reemplaza los módulos clásicos por convoluciones separables en profundidad (depthwise separable), lo que reduce drásticamente el número de parámetros sin sacrificar capacidad de representación. Gracias a esa eficiencia y a su entrenamiento previo en ImageNet, Xception se adapta muy bien a tareas médicas donde los conjuntos de datos son limitados, como la clasificación de imágenes de fondo de ojo para detectar Glaucoma.

In [8]:
# ────────────────────────── Bloque 0 – Librerías y entorno ──────────────────────────
import warnings, os, random, cv2, numpy as np, pandas as pd, matplotlib.pyplot as plt
warnings.filterwarnings("ignore")

import tensorflow as tf
from tensorflow.keras.applications import Xception
from tensorflow.keras.applications.xception import preprocess_input
from tensorflow.keras.layers import (Input, GlobalAveragePooling2D, Dense,
                                     Dropout, BatchNormalization)
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import (EarlyStopping, ReduceLROnPlateau,
                                        ModelCheckpoint)

from sklearn.model_selection import train_test_split
from sklearn.metrics import (confusion_matrix, classification_report,
                             precision_score, recall_score, f1_score, roc_auc_score)


In [9]:
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"  # 0=ALL, 1=INFO, 2=WARNING, 3=ERROR


In [10]:
# Configuración reproducible
SEED = 42
tf.keras.utils.set_random_seed(SEED)
np.random.seed(SEED)
random.seed(SEED)

# Verificación GPU
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    print(f"✅ GPU detectada: {gpus[0].name}")
    try:
        tf.config.experimental.set_memory_growth(gpus[0], True)
    except Exception as e:
        print(f"⚠️  No se pudo habilitar memory growth: {e}")
else:
    print("⚠️  No hay GPU, trabajaremos en CPU")

✅ GPU detectada: /physical_device:GPU:0


In [11]:
# Hiperparámetros globales
IMG_SIZE   = (299, 299)
BATCH      = 16
EPOCHS     = 30
LR         = 1e-4
VAL_SPLIT  = 0.10
TEST_SPLIT = 0.10
AUTOTUNE   = tf.data.AUTOTUNE
WORK_DIR   = r"G:\Mi unidad\Master_MIAA\1er_Semestre\5_ProyectoiNN\MIAA-ICESI-ProyectoIA\models"
USE_CLAHE  = True          # Activa o desactiva CLAHE
CLAHE_CLIP = 2.0           # Parámetro de contraste para CLAHE

## Descripción de las bases de datos:

1. ORIGA(-light) contiene 650 imágenes retinianas anotadas por profesionales formados del Instituto de Investigación Ocular de Singapur.

    La base de datos está compuesta por un conjunto de imágenes y un archivo en formato CSV que contiene las columnas que se aprecian a continuación:

    Las  columnas (Image, CDR, Ecc-Cup, Ecc-Disc)  corresponden a variables explicativas y la última (Glaucoma) corresponde a nuestra variable clasificatoria.

    Cada una de estas columnas se explica de la siguiente manera:


    - Image: Nombre de archivo de imagen.
    - Source: Fuente del dato
    - CDR: Cup-to-Disc Ratio. Proporción entre el diámetro del "cup" (excavación central) y el disco óptico. Valor clave en la detección de glaucoma..
    - Ecc-Cup: Excentricidad de la región de excavación del nervio óptico (cup). Una medida morfológica.
    - Ecc-Disc: Excentricidad del disco óptico completo. Ayuda a describir la forma del disco.
    - Glaucoma: Variable clasificatoria que identifica el diagnóstico negativo (0) o positivo (1) del glaucoma

2. G1020 esta base de datos consta de 1020 imágenes de fondo de ojo en color de alta resolución y proporciona anotaciones de la verdad fundamental para el diagnóstico del glaucoma.

    La base de datos está compuesta por un conjunto de imágenes y un archivo en formato CSV en donde se encuentra el label de la variable predictora (Glaucoma).

3. REFUGE (Retinal Fundus Glaucoma Challenge) es una base de datos pública creada para fomentar el desarrollo de algoritmos de inteligencia artificial en la detección automática de glaucoma a partir de imágenes de fondo de ojo (retinografías). Contiene 1200 imágenes de fondo de ojo (color fundus images) de alta calidad, provenientes de los centros clínicos Zhongshan Ophthalmic Center (ZOC) y Beijing Tongren Hospital (TR).

### Bases de datos públicas utilizadas para detección de Glaucoma

| Dataset  | Nº de Imágenes | Resolución | Anotaciones Disponibles | Etiqueta de Glaucoma | Fuente | Acceso |
|----------|----------------|------------|--------------------------|----------------------|--------|--------|
| **REFUGE** | 1200 | ~2124×2056 px | Segmentación de disco y copa óptica | ✅ | Zhongshan Ophthalmic Center, Beijing Tongren Hospital | [refuge.grand-challenge.org](https://refuge.grand-challenge.org/) |
| **ORIGA**  | 650  | 3072×2048 px   | Segmentación + CDR + labels | ✅ | Singapore Eye Research Institute | [Kaggle](https://www.kaggle.com/datasets/arnavjain1/glaucoma-datasets?select=ORIGA) |
| **G1020**  | 1020 | 2124×2056 px   | Segmentación + medidas clínicas (CDR, VCDR, etc.) | ✅ | MESSIDOR + anotación médica posterior | [G1020 en Zenodo](https://zenodo.org/record/6333984) |



## Carga de los datos

In [12]:
# ────────────────── Bloque 1 – Lectura / limpieza del DataFrame ──────────────────
# Tus lecturas originales (se asume que ya existen las variables):
#   df_origa, df_g1020, df_refuge_train

# Aseguramos columnas uniformes
df_g1020   = df_g1020.rename(columns={'imageID': 'Image', 'binaryLabels': 'Glaucoma'})
df_origa   = df_origa.rename(columns={'Label'    : 'Glaucoma'})
df_refuge_train = df_refuge_train.rename(columns={'ImgName': 'Image', 'Label': 'Glaucoma'})

# Fuente
df_origa['Source']        = 'ORIGA'
df_g1020['Source']        = 'G1020'
df_refuge_train['Source'] = 'REFUGE'

# Unificamos
df_glaucoma = pd.concat(
    [df_origa[['Image','Source','Glaucoma']],
     df_g1020[['Image','Source','Glaucoma']],
     df_refuge_train[['Image','Source','Glaucoma']]],
    ignore_index=True
)

# Añadimos columna Path
ROOT = r"G:\Mi unidad\Master_MIAA\1er_Semestre\5_ProyectoiNN\MIAA-ICESI-ProyectoIA\datos"
map_roots = {
    'ORIGA' : os.path.join(ROOT, 'ORIGA', 'Images'),
    'G1020' : os.path.join(ROOT, 'G1020', 'Images'),
    'REFUGE': os.path.join(ROOT, 'REFUGE', 'train', 'Images'),
}

def build_path(row):
    return os.path.join(map_roots.get(row['Source'], ''), row['Image'])

df_glaucoma['Path'] = df_glaucoma.apply(build_path, axis=1)

# Chequeo rápido
display(df_glaucoma.head())
print("Distribución de clases:\n", df_glaucoma['Glaucoma'].value_counts())


NameError: name 'df_g1020' is not defined

Bloque 2 – Split estratificado y pesos de clase

In [ ]:
# ────────────────────── Bloque 2 – Train / Val / Test Split ──────────────────────
train_df, temp_df = train_test_split(
    df_glaucoma,
    test_size=VAL_SPLIT + TEST_SPLIT,
    stratify=df_glaucoma['Glaucoma'],
    random_state=SEED
)

val_df, test_df = train_test_split(
    temp_df,
    test_size = TEST_SPLIT / (VAL_SPLIT + TEST_SPLIT),
    stratify  = temp_df['Glaucoma'],
    random_state=SEED
)

print(f"Train: {len(train_df)} | Val: {len(val_df)} | Test: {len(test_df)}")

# Pesos de clase para mitigar el desequilibrio
neg, pos = df_glaucoma['Glaucoma'].value_counts().sort_index().tolist()
total = neg + pos
class_weight = {
    0: total / (2 * neg),
    1: total / (2 * pos)
}
print("Pesos de clase:", class_weight)


Bloque 3 – Data pipeline con tf.data y (opcional) CLAHE + augmentations

In [ ]:
# ───────────── Bloque 3 ─────────────
def clahe_np(img, clip=CLAHE_CLIP):
    lab = cv2.cvtColor(img, cv2.COLOR_RGB2LAB)
    l, a, b = cv2.split(lab)
    clahe = cv2.createCLAHE(clipLimit=clip, tileGridSize=(8, 8))
    cl = clahe.apply(l)
    merged = cv2.merge((cl, a, b))
    rgb = cv2.cvtColor(merged, cv2.COLOR_LAB2RGB)
    return rgb

def process_path(path, label):
    img = tf.io.read_file(path)
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.image.convert_image_dtype(img, tf.float32)  # [0,1]

    if USE_CLAHE:
        img = tf.numpy_function(clahe_np, [tf.cast(img * 255.0, tf.uint8), CLAHE_CLIP], tf.uint8)
        img.set_shape([None, None, 3])            # ← recupera dims perdidas
        img = tf.image.convert_image_dtype(img, tf.float32)

    img = tf.image.resize(img, IMG_SIZE)
    return img, tf.cast(label, tf.float32)

def random_zoom(img, zoom_range=(0.9, 1.1)):
    zoom = tf.random.uniform([], zoom_range[0], zoom_range[1])
    new_size = tf.cast(tf.cast(IMG_SIZE, tf.float32) * zoom, tf.int32)
    img = tf.image.resize_with_crop_or_pad(img, new_size[0], new_size[1])
    return tf.image.resize(img, IMG_SIZE)

def augment(img, label):
    img = tf.image.random_flip_left_right(img)
    img = tf.image.random_brightness(img, 0.15)
    img = random_zoom(img)                       # ← zoom casero
    return img, label

train_ds = build_dataset(train_df, training=True)
val_ds   = build_dataset(val_df, training=False)
test_ds  = build_dataset(test_df, training=False)

Bloque 4 – Definición del modelo Xception + top layer

In [ ]:
# ───────────────────────── Bloque 4 – Arquitectura y compilación ─────────────────────────
inputs = Input(shape=(*IMG_SIZE, 3))
base   = Xception(weights='imagenet', include_top=False, input_tensor=inputs)
base.trainable = False  # Transfer learning congelado

x = GlobalAveragePooling2D()(base.output)
x = Dropout(0.3)(x)
x = Dense(128, activation='relu')(x)
x = BatchNormalization()(x)
outputs = Dense(1, activation='sigmoid')(x)

model = Model(inputs, outputs)
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=LR),
    loss='binary_crossentropy',
    metrics=['accuracy',
             tf.keras.metrics.AUC(name='auc'),
             tf.keras.metrics.Precision(name='precision'),
             tf.keras.metrics.Recall(name='recall')]
)

model.summary()


Bloque 5 – Callbacks y entrenamiento

In [ ]:
# ──────────────────────── Bloque 5 – Callbacks y entrenamiento ────────────────────────
os.makedirs(WORK_DIR, exist_ok=True)

callbacks = [
    EarlyStopping(monitor='val_auc', patience=5, mode='max', restore_best_weights=True),
    ReduceLROnPlateau(monitor='val_auc', factor=0.3, patience=3, mode='max', min_lr=1e-7),
    ModelCheckpoint(
        filepath=os.path.join(WORK_DIR, 'xception_glaucoma_best.h5'),
        monitor='val_auc', mode='max', save_best_only=True, verbose=1)
]

history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=EPOCHS,
    callbacks=callbacks,
    class_weight=class_weight,
    verbose=1
)


Bloque 6 – Curvas de entrenamiento

In [ ]:
# ────────────────────────── Bloque 6 – Curvas ACC / AUC ──────────────────────────
def plot_metric(metric_name):
    plt.figure()
    plt.plot(history.history[metric_name], label=f'Train {metric_name}')
    plt.plot(history.history[f'val_{metric_name}'], label=f'Val {metric_name}')
    plt.title(metric_name.upper())
    plt.xlabel('Epochs'); plt.ylabel(metric_name); plt.legend(); plt.grid()
    plt.show()

for m in ['accuracy', 'auc', 'precision', 'recall', 'loss']:
    plot_metric(m)


Bloque 7 – Evaluación en el set de prueba

In [ ]:
# ────────────────────────── Bloque 7 – Evaluación final ──────────────────────────
# 1. Métricas directas
test_loss, test_acc, test_auc, test_prec, test_rec = model.evaluate(test_ds, verbose=0)
test_f1 = 2 * (test_prec * test_rec) / (test_prec + test_rec + 1e-7)
print(f"\nTest  | Acc: {test_acc:.3f}  AUC: {test_auc:.3f}  "
      f"Prec: {test_prec:.3f}  Rec: {test_rec:.3f}  F1: {test_f1:.3f}")

# 2. Confusion matrix y reporte completo
y_true = np.concatenate([y for _, y in test_ds], axis=0)
y_pred_prob = model.predict(test_ds).ravel()
y_pred = (y_pred_prob >= 0.5).astype(int)

cm = confusion_matrix(y_true, y_pred)
print("\nMatriz de confusión:\n", cm)

print("\nClasificación detallada:\n",
      classification_report(y_true, y_pred, digits=3))

# ROC-AUC (sklearn)
print("ROC-AUC (sklearn):", roc_auc_score(y_true, y_pred_prob).round(3))
